## Problem2: Skip Connection Densenet121

In [1]:
import os 
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

In [3]:
class BuildDensenet121():
    def __init__(self):
        pass
    
    
    def conv_block(self, inputs, k=32):
        x = layers.BatchNormalization()(inputs)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(filters=4*k, kernel_size=(1,1), use_bias=False)(x)
        
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(filters=k, kernel_size=(3,3), padding='same', use_bias=False)(x)
        
        skip_connection = layers.Concatenate(axis=-1)([inputs,x])
        
        return skip_connection
    
    
    def dense_block(self, x, nums_block, k=32):
        for i in range(nums_block):
            x = self.conv_block(x, 32)
        return x
        
    
    
    def trasition_layer(self, inputs, fraction=0.5):
        channels = inputs.shape[-1]
        x = layers.BatchNormalization()(inputs)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(filters=fraction*channels, kernel_size=(1,1), use_bias=False)(x)
        x = layers.AveragePooling2D(2, strides=2)(x)
        
        return x
    
    def build_model(self, classes, input_shape=(224,224,3)):
        inputs = layers.Input(shape=input_shape)
        
        x = layers.ZeroPadding2D(padding=((3, 3)))(inputs)
        x = layers.Conv2D(64, 7, strides=2, use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.ZeroPadding2D(padding=((1, 1)))(x)
        x = layers.MaxPooling2D(3, strides=2)(x)
        
        # 1-6
        x = self.dense_block(x, nums_block=6)
        x = self.trasition_layer(x)
        
        # 2-12
        x = self.dense_block(x, nums_block=12)
        x = self.trasition_layer(x)
        
        # 3-24
        x = self.dense_block(x, nums_block=24)
        x = self.trasition_layer(x)
        
        #4-16
        x = self.dense_block(x, nums_block=16)
        
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.GlobalAveragePooling2D()(x)
        outputs = layers.Dense(classes, activation='softmax')(x)
        
        model = Model(inputs=inputs, outputs=outputs)
        
        return model

In [4]:
build_densenet121 = BuildDensenet121()
densenet121_model = build_densenet121.build_model(classes=1000)

2022-12-30 11:11:02.261082: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-30 11:11:02.287695: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-30 11:11:02.287772: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-30 11:11:02.288163: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
densenet121_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

 activation_8 (Activation)      (None, 56, 56, 128)  0           ['batch_normalization_8[0][0]']  
                                                                                                  
 conv2d_8 (Conv2D)              (None, 56, 56, 32)   36864       ['activation_8[0][0]']           
                                                                                                  
 concatenate_3 (Concatenate)    (None, 56, 56, 192)  0           ['concatenate_2[0][0]',          
                                                                  'conv2d_8[0][0]']               
                                                                                                  
 batch_normalization_9 (BatchNo  (None, 56, 56, 192)  768        ['concatenate_3[0][0]']          
 rmalization)                                                                                     
                                                                                                  
 activatio

                                                                  'conv2d_17[0][0]']              
                                                                                                  
 batch_normalization_18 (BatchN  (None, 28, 28, 192)  768        ['concatenate_7[0][0]']          
 ormalization)                                                                                    
                                                                                                  
 activation_18 (Activation)     (None, 28, 28, 192)  0           ['batch_normalization_18[0][0]'] 
                                                                                                  
 conv2d_18 (Conv2D)             (None, 28, 28, 128)  24576       ['activation_18[0][0]']          
                                                                                                  
 batch_normalization_19 (BatchN  (None, 28, 28, 128)  512        ['conv2d_18[0][0]']              
 ormalizat

                                                                                                  
 concatenate_12 (Concatenate)   (None, 28, 28, 352)  0           ['concatenate_11[0][0]',         
                                                                  'conv2d_27[0][0]']              
                                                                                                  
 batch_normalization_28 (BatchN  (None, 28, 28, 352)  1408       ['concatenate_12[0][0]']         
 ormalization)                                                                                    
                                                                                                  
 activation_28 (Activation)     (None, 28, 28, 352)  0           ['batch_normalization_28[0][0]'] 
                                                                                                  
 conv2d_28 (Conv2D)             (None, 28, 28, 128)  45056       ['activation_28[0][0]']          
          

                                                                                                  
 conv2d_37 (Conv2D)             (None, 28, 28, 32)   36864       ['activation_37[0][0]']          
                                                                                                  
 concatenate_17 (Concatenate)   (None, 28, 28, 512)  0           ['concatenate_16[0][0]',         
                                                                  'conv2d_37[0][0]']              
                                                                                                  
 batch_normalization_38 (BatchN  (None, 28, 28, 512)  2048       ['concatenate_17[0][0]']         
 ormalization)                                                                                    
                                                                                                  
 activation_38 (Activation)     (None, 28, 28, 512)  0           ['batch_normalization_38[0][0]'] 
          

...

 activation_76 (Activation)     (None, 14, 14, 128)  0           ['batch_normalization_76[0][0]'] 
                                                                                                  
 conv2d_76 (Conv2D)             (None, 14, 14, 32)   36864       ['activation_76[0][0]']          
                                                                                                  
 concatenate_36 (Concatenate)   (None, 14, 14, 864)  0           ['concatenate_35[0][0]',         
                                                                  'conv2d_76[0][0]']              
                                                                                                  
 batch_normalization_77 (BatchN  (None, 14, 14, 864)  3456       ['concatenate_36[0][0]']         
 ormalization)                                                                                    
                                                                                                  
 activatio

 ormalization)                                                                                    
                                                                                                  
 activation_86 (Activation)     (None, 14, 14, 128)  0           ['batch_normalization_86[0][0]'] 
                                                                                                  
 conv2d_86 (Conv2D)             (None, 14, 14, 32)   36864       ['activation_86[0][0]']          
                                                                                                  
 concatenate_41 (Concatenate)   (None, 14, 14, 1024  0           ['concatenate_40[0][0]',         
                                )                                 'conv2d_86[0][0]']              
                                                                                                  
 batch_normalization_87 (BatchN  (None, 14, 14, 1024  4096       ['concatenate_41[0][0]']         
 ormalizat

 conv2d_95 (Conv2D)             (None, 7, 7, 32)     36864       ['activation_95[0][0]']          
                                                                                                  
 concatenate_45 (Concatenate)   (None, 7, 7, 640)    0           ['concatenate_44[0][0]',         
                                                                  'conv2d_95[0][0]']              
                                                                                                  
 batch_normalization_96 (BatchN  (None, 7, 7, 640)   2560        ['concatenate_45[0][0]']         
 ormalization)                                                                                    
                                                                                                  
 activation_96 (Activation)     (None, 7, 7, 640)    0           ['batch_normalization_96[0][0]'] 
                                                                                                  
 conv2d_96

 activation_105 (Activation)    (None, 7, 7, 128)    0           ['batch_normalization_105[0][0]']
                                                                                                  
 conv2d_105 (Conv2D)            (None, 7, 7, 32)     36864       ['activation_105[0][0]']         
                                                                                                  
 concatenate_50 (Concatenate)   (None, 7, 7, 800)    0           ['concatenate_49[0][0]',         
                                                                  'conv2d_105[0][0]']             
                                                                                                  
 batch_normalization_106 (Batch  (None, 7, 7, 800)   3200        ['concatenate_50[0][0]']         
 Normalization)                                                                                   
                                                                                                  
 activatio

 Normalization)                                                                                   
                                                                                                  
 activation_115 (Activation)    (None, 7, 7, 128)    0           ['batch_normalization_115[0][0]']
                                                                                                  
 conv2d_115 (Conv2D)            (None, 7, 7, 32)     36864       ['activation_115[0][0]']         
                                                                                                  
 concatenate_55 (Concatenate)   (None, 7, 7, 960)    0           ['concatenate_54[0][0]',         
                                                                  'conv2d_115[0][0]']             
                                                                                                  
 batch_normalization_116 (Batch  (None, 7, 7, 960)   3840        ['concatenate_55[0][0]']         
 Normaliza